In [30]:
fbm_new.loc[fbm_new['平台订单号'].isin(fbm_merge['平台订单号']),'平台订单号'] = fbm_merge1['平台订单号']

In [1]:
import csv
import os
import pymysql
from pathlib import Path
from openpyxl import Workbook
import pandas as pd
import re
import numpy as np
import time

def read(basename, tablename):
    connc = pymysql.Connect(
        host='rm-bp15wb9k07q3p10q6yo.mysql.rds.aliyuncs.com',
        user='test1',
        password='123456',
        database='%s' % basename,
        port=3306,
        charset='utf8'
    )

    cur = connc.cursor()
    sql = 'select * from `%s`;' % tablename
    cur.execute(sql)

    des = cur.description
    title = []
    for j in range(len(des)):
        title.append(des[j][0])

    origin = cur.fetchall()
    origin = pd.DataFrame(list(origin))
    origin.columns = title
    return origin

In [3]:
fbm_new =  read('inventory','自发货数据new')
fbm =  read('inventory','自发货数据')
fbm_s =  read('inventory','自发货源数据')
fbm_sn = fbm_s[fbm_s['订单类型'].str.contains('正常订单')]
fbm_sd = fbm_s[fbm_s['订单类型'].str.contains('补寄订单')]
# inventory = pd.read_excel('E:/库存进度表/库存进度表4.17 - 新.xlsx',sheet_name = '库存清单')
fbm_new1 = fbm_new[fbm_new['平台订单号'].isin(fbm_sd['平台订单号'].drop_duplicates())]

In [18]:
outer = pd.merge(inventory['SKU'].drop_duplicates(), fbm_sn['SKU'].drop_duplicates(),on = 'SKU', how = 'outer')

now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time())) 
outer.to_excel(excel_writer="E:/库存进度表/"+now+r"sku表.xlsx", index=False,encoding='utf-8')

In [17]:
fbm_sd.head()

,SKU,数量,订单类型,发货时间,账号,平台订单号,包裹号,平台
516,USAN1019185-1,1,补寄订单,2021-02-26 04:39:50.0,amazon宫本,113-9092551-8217040,AK2021022300252,amazon
551,USAN1015827-1,1,补寄订单,2021-02-25 10:30:55.0,amazon宫本,112-3469255-3680218,AK2021021700224,amazon
555,USAN1018111,1,补寄订单,2021-02-25 10:12:55.0,amazon宫本,111-4402262-2573003,AK2021022300226,amazon
581,USAN1019197,1,补寄订单,2021-02-25 06:27:51.0,amazon宫本,112-6516215-2416255,AK2021021600263,amazon
583,USAN1019210-1,1,补寄订单,2021-02-25 06:07:51.0,amazon宫本,112-5048717-5720221,AK2021022200247,amazon


In [8]:
# fbm_sd_pivot = fbm_sd.pivot_table(index = ['平台订单号','SKU'],aggfunc = {'数量':'sum'})
# fbm_sd_pivot = fbm_sd_pivot.reset_index()
# fbm_sd_pivot

In [9]:
# bag_merge = pd.merge(fbm_source_duplicate_pivot,fbm_source_duplicate.drop('数量',axis = 1), on = ['平台订单号','SKU'], )
# bag_merge[bag_merge['平台订单号'] == '8863666102551']


In [10]:
# fbm_source_duplicate_pivot[fbm_source_duplicate_pivot['平台订单号'] == '8863666102551']

In [11]:
# fbm_sd_pivot = fbm_sd.pivot_table(index = ['平台订单号','包裹号'],aggfunc = {'SKU':'count'})
# fbm_sd_pivot = fbm_sd_pivot.reset_index()
# # fbm_sd_pivot = fbm_sd_pivot.pivot_table(index = ['平台订单号'],aggfunc = {'包裹号':'count'})
# # fbm_sd_pivot = fbm_sd_pivot.reset_index()
# fbm_sd_pivot.head()

In [12]:
# fbm_sd_pivot[fbm_sd_pivot['平台订单号'] == '8863666102551']

In [13]:
fbm_sd[fbm_sd['平台订单号'] == '8863666102551']

,SKU,数量,订单类型,发货时间,账号,平台订单号,包裹号,平台
929267,USAN1032308,2,补寄订单,2022-01-30 11:22:25.0,walmart优瑞斯特,8863666102551,AK2022012708724,walmart
929268,USAN1032312,1,补寄订单,2022-01-30 11:20:29.0,walmart优瑞斯特,8863666102551,AK2022012708725,walmart
929269,USAN1032307,1,补寄订单,2022-01-30 11:20:29.0,walmart优瑞斯特,8863666102551,AK2022012708725,walmart
1055570,USAN1032311,2,补寄订单,2022-02-22 19:13:32.0,walmart优瑞斯特,8863666102551,AK2022021107693,walmart


In [21]:
for i in range(len(fbm_new)):
    order_list = fbm_sd[fbm_sd['平台订单号'] == fbm_new.iloc[i,0]]
#     order_sit = fbm_sd_pivot['平台订单号'].tolist().index(fbm_new.iloc[i,0])
    if  order_list.empty:
        continue
    else:
        sku_num = 0
        for j in range(len(order_list)):
            if order_list.iloc[j,0] in outer:
                sku_num = sku_num + order_list.iloc[j,1]
        fbm.loc[i,'sku补寄数量'] = sku_num
#         fbm_new.loc[i,'配件补寄次数'] = fbm_sd_pivot.iloc[order_sit,1]

In [22]:
fbm

,平台订单号,账号,SKU,数量,补寄数量,最早时间,最晚时间,sku补寄数量
0,01-07606-52761,ebay玲琅,USAN1023893-2,1.0,NaN,2021-09-16 03:08:01.0,2021-09-16 03:08:01.0,NaN
1,01-07620-34265,ebay玲琅,USAN1029590-1,1.0,NaN,2021-09-20 23:35:05.0,2021-09-20 23:35:05.0,NaN
2,01-07623-11933,ebay玲琅,USAN1011332-1,1.0,NaN,2021-09-24 11:10:38.0,2021-09-24 11:10:38.0,NaN
3,01-07627-40897,ebay玲琅,USAN1023811-2,1.0,NaN,2021-09-21 19:16:57.0,2021-09-21 19:16:57.0,NaN
4,01-07632-99344,ebay玲琅,USAN1017139-2,1.0,NaN,2021-09-24 11:06:32.0,2021-09-24 11:06:32.0,NaN
...,...,...,...,...,...,...,...,...
1583284,NF10834862664,shopifynextfur,USAN1020594-1,NaN,1.0,2022-03-08 21:39:41.0,2022-03-08 21:39:41.0,NaN
1583285,NF10834862708,shopifynextfur,USAN1023384-1,NaN,1.0,2022-03-17 00:57:11.0,2022-03-17 00:57:11.0,NaN
1583286,NF10834863073,shopifynextfur,USAN1023384-2,NaN,1.0,2022-04-02 02:08:18.0,2022-04-02 02:08:18.0,NaN
1583287,NF10834863323,shopifynextfur,USAN1034654-7,NaN,1.0,2022-03-31 05:54:02.0,2022-03-31 05:54:02.0,NaN


In [107]:
order_list = fbm_sd[fbm_sd['平台订单号'] == fbm_new.iloc[1284635,0]]
order_list

,SKU,数量,订单类型,发货时间,账号,平台订单号,包裹号,平台
929267,USAN1032308,2,补寄订单,2022-01-30 11:22:25.0,walmart优瑞斯特,8863666102551,AK2022012708724,walmart
929268,USAN1032312,1,补寄订单,2022-01-30 11:20:29.0,walmart优瑞斯特,8863666102551,AK2022012708725,walmart
929269,USAN1032307,1,补寄订单,2022-01-30 11:20:29.0,walmart优瑞斯特,8863666102551,AK2022012708725,walmart
1055570,USAN1032311,2,补寄订单,2022-02-22 19:13:32.0,walmart优瑞斯特,8863666102551,AK2022021107693,walmart


In [133]:
order_sit = fbm_sd_pivot['平台订单号'].tolist().index(fbm_new.iloc[1284635,0])
order_sit

14037

In [134]:
for j in range(len(order_list)):
    if order_list.iloc[j,0] in outer:
        sku_num = sku_num + order_list.iloc[j,1]
fbm_new.loc[1284635,'sku补寄数量'] = sku_num
# fbm_new.loc[1284635,'配件补寄次数'] = fbm_sd_pivot.iloc[order_sit,1]

In [135]:
fbm_new[fbm_new['平台订单号'] == '8863666102551']

,平台订单号,账号,数量,补寄数量,最早时间,最晚时间,sku补寄数量,配件补寄次数
1284635,8863666102551,walmart优瑞斯特,1.0,6.0,2022-01-05 05:18:28.0,2022-02-22 19:13:32.0,0.0,3.0


0

In [1]:
import csv
import os
import pymysql
from pathlib import Path
from openpyxl import Workbook
import pandas as pd
import re
import numpy as np
import time

def read(basename, tablename):
    connc = pymysql.Connect(
        host='rm-bp15wb9k07q3p10q6yo.mysql.rds.aliyuncs.com',
        user='test_admin',
        password='123456',
        database='%s' % basename,
        port=3306,
        charset='utf8'
    )

    cur = connc.cursor()
    sql = 'select * from `%s`;' % tablename
    cur.execute(sql)

    des = cur.description
    title = []
    for j in range(len(des)):
        title.append(des[j][0])

    origin = cur.fetchall()
    origin = pd.DataFrame(list(origin))
    origin.columns = title
    return origin

In [93]:
fbm_new =  read('inventory','自发货源数据new1')


In [207]:
fbm_new1 = fbm_new.copy()
fbm_new1['平台订单号'] =fbm_new1['平台订单号'].astype(str)

In [208]:
walmart = pd.read_excel(r'D:/work/优瑞斯特2021订单 6-7及之前.xlsx')
walmart['PO#'] = walmart['PO#'].astype(str)
walmart['Order#'] = walmart['Order#'].astype(str)
print(len(walmart))
walmart.head(2)

672


,PO#,Order#,平台sku,公司sku,品名,小组,年,月,日,Order Date,...,Tax,Update Status,Update Qty,Carrier,Tracking Number,Tracking Url,Seller Order NO,Fulfillment Entity,Replacement Order,Original Customer Order Id
0,7811227436175,3922163128015,WM1018839-5,USAN1018839-5,55“深色胡桃-黑色框架,A,2021,4,30,2021-04-30,...,6.6,NaN,NaN,NaN,786672755716,https://www.walmart.com/tracking?tracking_id=7...,NaN,Seller Fulfilled,No,NaN
1,7811227436175,3922163128015,WM1018839-5,USAN1018839-5,55“深色胡桃-黑色框架,A,2021,4,30,2021-04-30,...,6.6,NaN,NaN,NaN,786672755966,https://www.walmart.com/tracking?tracking_id=7...,NaN,Seller Fulfilled,No,NaN


In [209]:
fbm_new.dtypes

平台订单号                object
账号                   object
平台SKU                object
公司SKU                object
数量                  float64
包裹号                  object
订单状态                 object
订单时间                 object
补寄sku数量             float64
补寄sku订单时间            object
补寄配件次数              float64
补寄配件订单时间     datetime64[ns]
dtype: object

In [210]:
walmart = walmart[['PO#','Order#','公司sku']]
walmart = walmart.drop_duplicates()
walmart = walmart.reset_index(drop= True)
walmart

,PO#,Order#,公司sku
0,7811227436175,3922163128015,USAN1018839-5
1,1814917746062,3922163138900,USAN1019552-9
2,2811217124831,3912162273949,USAN1018799-3
3,1814907268687,3912162842000,USAN1018839-1
4,6814907007468,3912161588162,USAN1011322-2
...,...,...,...
636,1815249701327,4252113475350,USAN1023426-1
637,1815249685409,4252113360048,USAN1011322-5
638,1811559347791,4252113773799,PTG103482-01
639,2811559332795,4252113566749,EXE21050400006


In [211]:
fbm_new1[fbm_new1['平台订单号']=='3922163138900']

,平台订单号,账号,平台SKU,公司SKU,数量,包裹号,订单状态,订单时间,补寄sku数量,补寄sku订单时间,补寄配件次数,补寄配件订单时间
1661607,3922163138900,walmart优瑞斯特,WM1019552-9,USAN1019552-9,1.0,AK21050302432,已发货,2021-05-05 11:39:28,NaN,None,NaN,NaT


In [212]:
order_list = fbm_new1[fbm_new1['平台订单号'].isin(walmart['Order#'])]
order_list

,平台订单号,账号,平台SKU,公司SKU,数量,包裹号,订单状态,订单时间,补寄sku数量,补寄sku订单时间,补寄配件次数,补寄配件订单时间
1540883,4252113150267,walmart优瑞斯特,WM1011322-5,USAN1011322-5,1.0,AK2021060303565,已发货,2021-06-03 20:31:00,NaN,None,NaN,NaT
1540884,4252113045347,walmart优瑞斯特,WM1023361-3,USAN1023361-3,1.0,AK2021060303688,已发货,2021-06-03 21:52:00,NaN,None,NaN,NaT
1540885,4252113340504,walmart优瑞斯特,WM1017103-3-4,USAN1026771-3,1.0,AK2021060303535,已发货,2021-06-03 20:00:00,NaN,None,NaN,NaT
1540886,4252113340504,walmart优瑞斯特,WM1017103-3-4,USAN1026771-4,1.0,AK2021060403372,已发货,2021-06-04 10:33:00,NaN,None,NaN,NaT
1540887,4252113442728,walmart优瑞斯特,WM103484,HOG103484,1.0,AK2021060303602,已发货,2021-06-03 20:36:00,NaN,None,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
1661858,4192104967696,walmart优瑞斯特,WM1023361-3,USAN1023361-3,1.0,AK21052811055,已发货,2021-05-29 01:20:08,NaN,None,NaN,NaT
1661859,4192104349341,walmart优瑞斯特,WM1011322-5,USAN1011322-5,1.0,AK21052811056,已发货,2021-05-29 02:20:22,NaN,None,NaN,NaT
1661860,4202106154990,walmart优瑞斯特,WM1011322-2,USAN1011322-2,1.0,AK21053114411,已发货,2021-05-31 21:20:25,NaN,None,NaN,NaT
1661861,4202106717947,walmart优瑞斯特,WM1011322-2,USAN1011322-2,1.0,AK21053114408,已发货,2021-05-31 21:20:25,NaN,None,NaN,NaT


In [213]:
for i in range(len(walmart)):
    if walmart.iat[i,1] in order_list['平台订单号'].tolist():
        print(walmart.iat[i,1])
        walmart_sit = walmart['Order#'].tolist().index(walmart.iat[i,1])
        print(walmart_sit)
        fbm_new1.loc[fbm_new1['平台订单号'] ==walmart.iat[i,1],'平台订单号' ]= walmart.loc[walmart_sit,'PO#']
        print(fbm_new1.loc[fbm_new1['平台订单号'] ==walmart.iat[i,1] ],'asd')

3922163128015
0
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
3922163138900
1
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
3912162273949
2
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
3912162842000
3
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
3912161588162
4
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
3902161941075
5
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
3902161424920
6
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
3902160494757
7
Empt

Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4012176655727
60
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4012176234247
61
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4012176426757
62
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4012176818389
63
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4012176113899
64
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4012176106911
65
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4012176602369
66
Empty DataFra

Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4222108940124
119
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4222108945715
120
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4222108042665
121
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4222108353200
122
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4222108238728
123
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4222108948339
124
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4222108847724
125
Empty 

Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4202106154990
177
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4202106047722
178
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4202106442854
179
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4202106543623
180
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4202106445742
181
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4202106235162
182
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4202106719306
184
Empty 

Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4162101375743
237
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4162101055003
238
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4162101255293
239
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4162101152817
240
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4162101730535
241
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4162101631201
242
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4162101519217
243
Empty 

Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4122196495107
296
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4132197104728
297
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4122196577786
298
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4122196464197
299
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4122196759746
300
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4122196258760
301
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4122196652682
302
Empty 

Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4082186010179
356
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4082186103223
357
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
8406217419850
358
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4082185996204
359
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4082185399760
360
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4072185787447
361
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4082185695883
362
Empty 

Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4242112393551
421
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4242112484533
422
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4242112181329
423
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4242112267188
424
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4242112459853
425
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4242112263055
426
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4242112268560
427
Empty 

Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4282117677285
531
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4282117679929
532
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4282117787352
533
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4282117872403
534
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4282117469700
535
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4282117464561
536
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4282117375614
537
Empty 

Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4262115639376
593
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4262115322300
594
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4262115408437
595
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4262114499485
596
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4262115913126
597
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4262115606440
598
Empty DataFrame
Columns: [平台订单号, 账号, 平台SKU, 公司SKU, 数量, 包裹号, 订单状态, 订单时间, 补寄sku数量, 补寄sku订单时间, 补寄配件次数, 补寄配件订单时间]
Index: [] asd
4262114290058
599
Empty 

In [ ]:
fbm_new.loc[1661436,'平台订单号']

In [214]:
fbm_new1.loc[fbm_new1['平台订单号'] =='3922163138900' ]

,平台订单号,账号,平台SKU,公司SKU,数量,包裹号,订单状态,订单时间,补寄sku数量,补寄sku订单时间,补寄配件次数,补寄配件订单时间


In [217]:
# fbm_new_po = fbm_new[fbm_new['平台订单号'].isin(walmart['PO#'])]
# print(len(fbm_new_po))
# fbm_new_po
fbm_merge = pd.merge(fbm_new1,walmart,left_on = '平台订单号', right_on = 'Order#',how = 'inner')
fbm_merge = fbm_merge.drop_duplicates()
fbm_merge

,平台订单号,账号,平台SKU,公司SKU,数量,包裹号,订单状态,订单时间,补寄sku数量,补寄sku订单时间,补寄配件次数,补寄配件订单时间,PO#,Order#,公司sku


In [24]:
# qwe = pd.pivot_table(fbm_merge,index='平台订单号',values = '公司SKU',aggfunc ='count')
# qwe= qwe.reset_index()
# qwe
fbm_merge['平台订单号'].value_counts()

4282117845815    9
3852152079881    8
4272116180895    6
4092189562367    6
4142198382576    4
                ..
4232108484496    1
4232108978563    1
4222107290901    1
4222107790056    1
4192105326191    1
Name: 平台订单号, Length: 565, dtype: int64

In [37]:
fbm_merge1 =fbm_merge.copy()

In [38]:
fbm_merge1['平台订单号'] = fbm_merge1['PO#'] 

In [39]:
fbm_merge1

,平台订单号,账号,平台SKU,公司SKU,数量,包裹号,订单状态,订单时间,补寄sku数量,补寄sku订单时间,补寄配件次数,补寄配件订单时间,PO#,Order#,公司sku
0,1811559227881,walmart优瑞斯特,WM1011322-5,USAN1011322-5,1.0,AK2021060303565,已发货,2021-06-03 20:31:00,NaN,None,NaN,NaT,1811559227881,4252113150267,USAN1011322-5
1,4811559258065,walmart优瑞斯特,WM1023361-3,USAN1023361-3,1.0,AK2021060303688,已发货,2021-06-03 21:52:00,NaN,None,NaN,NaT,4811559258065,4252113045347,USAN1023361-3
2,2815249534613,walmart优瑞斯特,WM1017103-3-4,USAN1026771-3,1.0,AK2021060303535,已发货,2021-06-03 20:00:00,NaN,None,NaN,NaT,2815249534613,4252113340504,EXE21050400005
3,2815249534613,walmart优瑞斯特,WM1017103-3-4,USAN1026771-4,1.0,AK2021060403372,已发货,2021-06-04 10:33:00,NaN,None,NaN,NaT,2815249534613,4252113340504,EXE21050400005
4,1815249566215,walmart优瑞斯特,WM103484,HOG103484,1.0,AK2021060303602,已发货,2021-06-03 20:36:00,NaN,None,NaN,NaT,1815249566215,4252113442728,HOG103484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,2815185625277,walmart优瑞斯特,WM1023361-3,USAN1023361-3,1.0,AK21052811055,已发货,2021-05-29 01:20:08,NaN,None,NaN,NaT,2815185625277,4192104967696,USAN1023361-3
747,1811495244868,walmart优瑞斯特,WM1011322-5,USAN1011322-5,1.0,AK21052811056,已发货,2021-05-29 02:20:22,NaN,None,NaN,NaT,1811495244868,4192104349341,USAN1011322-5
748,2815206703270,walmart优瑞斯特,WM1011322-2,USAN1011322-2,1.0,AK21053114411,已发货,2021-05-31 21:20:25,NaN,None,NaN,NaT,2815206703270,4202106154990,USAN1011322-2
749,4811506154769,walmart优瑞斯特,WM1011322-2,USAN1011322-2,1.0,AK21053114408,已发货,2021-05-31 21:20:25,NaN,None,NaN,NaT,4811506154769,4202106717947,USAN1011322-2


In [28]:
fbm_merge1 = fbm_merge1[['平台订单号','账号','平台SKU','公司SKU','数量','包裹号','订单状态','订单时间','补寄sku数量','补寄sku订单时间','补寄配件次数','补寄配件订单时间']]


In [215]:
fbm_new[fbm_new['平台订单号'].isin(fbm_merge['平台订单号'])]['平台订单号'].value_counts()

4142198382576    4
4282118305795    4
4242111956244    4
3852152079881    4
4272116180895    3
                ..
4232108484496    1
4232108978563    1
4222107290901    1
4222107790056    1
4192105326191    1
Name: 平台订单号, Length: 565, dtype: int64

In [30]:
len(fbm_new.loc[fbm_new['平台订单号'].isin(fbm_merge['平台订单号']),'平台订单号'])

669

In [31]:
len(fbm_merge1['平台订单号'])

686

In [30]:
fbm_new.loc[fbm_new['平台订单号'].isin(fbm_merge['平台订单号']),'平台订单号'] = fbm_merge1['平台订单号']

In [31]:
fbm_new[fbm_new['平台订单号'] =='1811559227881']

,平台订单号,账号,平台SKU,公司SKU,数量,包裹号,订单状态,订单时间,补寄sku数量,补寄sku订单时间,补寄配件次数,补寄配件订单时间


In [41]:
qwe = fbm_new[fbm_new['平台订单号'].isnull()]
qwe

,平台订单号,账号,平台SKU,公司SKU,数量,包裹号,订单状态,订单时间,补寄sku数量,补寄sku订单时间,补寄配件次数,补寄配件订单时间
1522082,NaN,walmart优瑞斯特,WM1011322-5,USAN1011322-5,1.0,AK2021060303565,已发货,2021-06-03 20:31:00,NaN,None,NaN,NaT
1522083,NaN,walmart优瑞斯特,WM1023361-3,USAN1023361-3,1.0,AK2021060303688,已发货,2021-06-03 21:52:00,NaN,None,NaN,NaT
1522084,NaN,walmart优瑞斯特,WM1017103-3-4,USAN1026771-3,1.0,AK2021060303535,已发货,2021-06-03 20:00:00,NaN,None,NaN,NaT
1522085,NaN,walmart优瑞斯特,WM1017103-3-4,USAN1026771-4,1.0,AK2021060403372,已发货,2021-06-04 10:33:00,NaN,None,NaN,NaT
1522086,NaN,walmart优瑞斯特,WM103484,HOG103484,1.0,AK2021060303602,已发货,2021-06-03 20:36:00,NaN,None,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
1633126,NaN,walmart优瑞斯特,WM1023361-3,USAN1023361-3,1.0,AK21052811055,已发货,2021-05-29 01:20:08,NaN,None,NaN,NaT
1633127,NaN,walmart优瑞斯特,WM1011322-5,USAN1011322-5,1.0,AK21052811056,已发货,2021-05-29 02:20:22,NaN,None,NaN,NaT
1633128,NaN,walmart优瑞斯特,WM1011322-2,USAN1011322-2,1.0,AK21053114411,已发货,2021-05-31 21:20:25,NaN,None,NaN,NaT
1633129,NaN,walmart优瑞斯特,WM1011322-2,USAN1011322-2,1.0,AK21053114408,已发货,2021-05-31 21:20:25,NaN,None,NaN,NaT


In [84]:
from sqlalchemy import create_engine
connect1 = create_engine(
    f'mysql+pymysql://{"test1"}:{"123456"}@{"rm-bp15wb9k07q3p10q6yo.mysql.rds.aliyuncs.com"}:{3306}/{"inventory"}?charset=utf8')
fbm_new.to_sql("自发货源数据new1", connect1, if_exists='replace', index=False)


1666258